In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests10 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000

skip_shi = False
refinement_test=True
adapt_c = True

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )


In [5]:
# Input arrays
alpha = [0.005,.01,.025,.05]
results = np.array([
    [[0.98, 0.02, 0.00], [0.92, 0.08, 0.00]],
    [[0.86, 0.14, 0.00], [0.66, 0.34, 0.00]],
    [[0.66, 0.34, 0.00], [0.50, 0.50, 0.00]],
    [[0.54, 0.46, 0.00], [0.41, 0.59, 0.00]]
])
# Column headers

def print_mc2(alpha_levels ,test_results ):
    table = "\\begin{tabular}{c|cccc}\n"
    table += "\\hline\n"
    table += "\\textbf{$\\alpha$} & {} & \\textbf{No selection} & \\textbf{Model 1} & \\textbf{Model 2} \\\\\n"
    table += "\\hline\n"

    for i in range(len(alpha_levels)):
        alpha = alpha_levels[i]
        table += "{$%.2f$} & \\textbf{Normal} & %.2f & %.2f & %.2f \\\\\n" % (alpha, test_results[i][0][0], test_results[i][0][1], test_results[i][0][2])
        table += "& \\textbf{Bootstrap-ND} & %.2f & %.2f & %.2f \\\\\n" % (test_results[i][1][0], test_results[i][1][1], test_results[i][1][2])
        table += "& \\textbf{Shi (2015)} & %.2f & %.2f & %.2f \\\\\n" % (test_results[i][2][0], test_results[i][2][1], test_results[i][2][2])
        table += "\\hline\n"

    table += "\\end{tabular}"

    print(table)

#print_mc2(alpha,results)

# evidence of power

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:
import vuong_tests5

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)

res0 = vuong_tests5.monte_carlo(1,gen_data_ex,setup_shi_ex,trials=500,biascorrect=False)

nobs=250
a1,a2 = np.sqrt(1.09-1), 0.00
c1,c2 = calc_c(nobs)
print(c1)

cstar 64
64 cstar 25.585172603159354
64 cstar 25.585172603159354
64 cstar 25.585172603159354
0.0006324555320336759


# a  = .25, k= 4,n=500

In [8]:
num_params=4
nobs=500
a1,a2 = np.sqrt(1.09**.5-1), 0.00
c1,c2 = calc_c(nobs)
alphas =  [0.005,.01,.025,.05]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,
                                          data_tuned_c=data_tuned_c,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3,shi])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.71, 5th: 1.36, 10th: 1.91, 30th: 3.42, 50th: 4.90, 70th: 6.73, 90th: 10.07, 95th: 11.80, 99th: 16.53
bonus term 0.05122964676653753 // variance stats 0.30030530249906606 //part1 3.465724215775732 //part2 0.014781801313948694
-2.5634560165429967 3.3653673060893703 3.275956271986303
-2.6659153100760715 3.3653673060893703 3.1734969784532283
---
Quantiles: 1th: 0.65, 5th: 1.25, 10th: 1.69, 30th: 3.05, 50th: 4.40, 70th: 6.23, 90th: 9.41, 95th: 11.31, 99th: 15.87
bonus term 0.05350347114542819 // variance stats 0.2683201930105388 //part1 3.465724215775732 //part2 0.01543789055744371
-2.983590136380042 2.807554026872158 3.2566109071837364
-3.090597078670898 2.807554026872158 3.14960396489288
---
Quantiles: 1th: 0.64, 5th: 1.23, 10th: 1.72, 30th: 3.17, 50th: 4.65, 70th: 6.42, 90th: 9.92, 95th: 12.10, 99th: 16.57
bonus term 0.06079360671696671 // variance stats 0.24844742987468874 //part1 3.465724215775732 //part2 0.017541386138065603
-2.5070032599320204 2.6197943989018344 2.9

Quantiles: 1th: 0.62, 5th: 1.16, 10th: 1.62, 30th: 2.94, 50th: 4.25, 70th: 5.99, 90th: 9.28, 95th: 11.68, 99th: 15.57
bonus term 0.07610277503941427 // variance stats 0.18485030916288084 //part1 3.465724215775732 //part2 0.021958693277728165
-2.236892699606257 1.6031369103637103 2.856646437308962
-2.389098249685085 1.6031369103637103 2.7044408872301338
---
Quantiles: 1th: 0.55, 5th: 1.11, 10th: 1.55, 30th: 2.88, 50th: 4.22, 70th: 5.92, 90th: 9.14, 95th: 11.10, 99th: 15.45
bonus term 0.09208323804707234 // variance stats 0.14975246336204812 //part1 3.465724215775732 //part2 0.02656969577322856
-2.573287230051812 1.1839264741697997 2.6707503328405537
-2.7574537061459568 1.1839264741697997 2.486583856746409
---
Quantiles: 1th: 0.62, 5th: 1.10, 10th: 1.54, 30th: 2.76, 50th: 4.05, 70th: 5.66, 90th: 8.61, 95th: 10.30, 99th: 14.09
bonus term 0.06434144684903306 // variance stats 0.20357252517820956 //part1 3.465724215775732 //part2 0.018565079863006796
-2.417276034961022 1.7081602331785803 2.

Quantiles: 1th: 0.62, 5th: 1.17, 10th: 1.61, 30th: 2.87, 50th: 4.18, 70th: 5.89, 90th: 9.17, 95th: 10.85, 99th: 15.85
bonus term 0.0654013023529222 // variance stats 0.2132594993442037 //part1 3.465724215775732 //part2 0.018870890550154017
-1.9441977645176258 2.570890625101769 2.2606892527865625
-2.0750003692234706 2.570890625101769 2.129886648080718
---
Quantiles: 1th: 0.52, 5th: 1.07, 10th: 1.48, 30th: 2.78, 50th: 4.04, 70th: 5.61, 90th: 8.54, 95th: 10.23, 99th: 14.06
bonus term 0.07541540629873895 // variance stats 0.17172762722158763 //part1 3.465724215775732 //part2 0.02176035991422899
-2.17063654074941 2.15520855606046 2.224766085129138
-2.3214673533468884 2.15520855606046 2.07393527253166
---
Quantiles: 1th: 0.63, 5th: 1.21, 10th: 1.62, 30th: 2.92, 50th: 4.25, 70th: 5.96, 90th: 9.26, 95th: 10.97, 99th: 15.69
bonus term 0.0696492350934195 // variance stats 0.20196718484708379 //part1 3.465724215775732 //part2 0.020096588983157138
-1.873012707325109 1.9020610488518856 2.2615948711

Quantiles: 1th: 0.58, 5th: 1.21, 10th: 1.62, 30th: 3.10, 50th: 4.53, 70th: 6.33, 90th: 9.86, 95th: 11.93, 99th: 16.79
bonus term 0.07659833291884734 // variance stats 0.19511450989922352 //part1 3.465724215775732 //part2 0.022101681538934093
-1.9018895888224756 2.035565588830036 2.092248414841715
-2.05508625466017 2.035565588830036 1.9390517490040202
---
Quantiles: 1th: 0.59, 5th: 1.16, 10th: 1.58, 30th: 2.89, 50th: 4.28, 70th: 5.95, 90th: 9.17, 95th: 10.93, 99th: 15.83
bonus term 0.08855857312790973 // variance stats 0.1563404218670842 //part1 3.465724215775732 //part2 0.02555268902378249
-1.8501985377787737 1.9768174285776035 2.0446410821081233
-2.027315684034593 1.9768174285776035 1.8675239358523037
---
Quantiles: 1th: 0.57, 5th: 1.15, 10th: 1.62, 30th: 2.98, 50th: 4.34, 70th: 6.02, 90th: 9.25, 95th: 11.25, 99th: 16.40
bonus term 0.1017500713767691 // variance stats 0.13682067432072081 //part1 3.465724215775732 //part2 0.029358963680263406
-1.7009830187930453 1.8096495497997909 2.13

# a  = .25, k= 4, n=250

In [9]:
num_params=4
nobs=250
a1,a2 = np.sqrt(1.09-1), 0.00
c1,c2 = calc_c(nobs)
alphas = [0.005,.01,.025,.05]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,
                                          data_tuned_c=data_tuned_c,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3,shi])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.75, 5th: 1.34, 10th: 1.79, 30th: 3.17, 50th: 4.55, 70th: 6.27, 90th: 9.71, 95th: 11.52, 99th: 16.04
bonus term 0.044694034769677066 // variance stats 0.4082477376437466 //part1 3.017088168272582 //part2 0.014813632309348918
-2.9278881356884003 3.6444620694057175 3.113222849575314
-3.0172762052277546 3.6444620694057175 3.0238347800359597
---
Quantiles: 1th: 0.64, 5th: 1.15, 10th: 1.56, 30th: 2.87, 50th: 4.20, 70th: 5.79, 90th: 8.94, 95th: 10.96, 99th: 15.55
bonus term 0.04527766046743705 // variance stats 0.37095170044580544 //part1 3.017088168272582 //part2 0.015007072363205264
-3.0244629339084543 2.3583548209002805 3.029310570076777
-3.1150182548433287 2.3583548209002805 2.9387552491419027
---
Quantiles: 1th: 0.71, 5th: 1.35, 10th: 1.87, 30th: 3.28, 50th: 4.65, 70th: 6.62, 90th: 10.14, 95th: 12.13, 99th: 16.80
bonus term 0.05367052494048407 // variance stats 0.35407080220348786 //part1 3.017088168272582 //part2 0.01778884870017334
-2.5012314310889874 2.19267609005164

Quantiles: 1th: 0.63, 5th: 1.13, 10th: 1.51, 30th: 2.66, 50th: 3.73, 70th: 5.16, 90th: 8.03, 95th: 9.55, 99th: 13.40
bonus term 0.04890476796760885 // variance stats 0.3082227303248692 //part1 3.017088168272582 //part2 0.016209260465732102
-2.587524314348664 2.4799395424310298 2.7462812598115116
-2.685333850283882 2.4799395424310298 2.648471723876294
---
Quantiles: 1th: 0.67, 5th: 1.20, 10th: 1.60, 30th: 2.92, 50th: 4.17, 70th: 5.78, 90th: 8.82, 95th: 10.90, 99th: 14.61
bonus term 0.048750142532703565 // variance stats 0.33982902053367253 //part1 3.017088168272582 //part2 0.016158010576341628
-2.9008294131518775 2.673663593752852 2.8552320588924203
-2.9983296982172845 2.673663593752852 2.7577317738270133
---
Quantiles: 1th: 0.71, 5th: 1.34, 10th: 1.80, 30th: 3.51, 50th: 5.10, 70th: 7.19, 90th: 11.38, 95th: 13.90, 99th: 19.51
bonus term 0.08598021985604654 // variance stats 0.2453697190048122 //part1 3.017088168272582 //part2 0.028497748511364874
-2.509190389998724 1.7197483009764398 2.

Quantiles: 1th: 0.69, 5th: 1.28, 10th: 1.72, 30th: 3.06, 50th: 4.33, 70th: 6.02, 90th: 9.35, 95th: 11.46, 99th: 15.69
bonus term 0.05200747516434157 // variance stats 0.3370400921986588 //part1 3.017088168272582 //part2 0.01723763849901615
-2.0647400492088357 1.486986214226388 2.4962441243760747
-2.1687549995375193 1.486986214226388 2.392229174047391
---
Quantiles: 1th: 0.67, 5th: 1.23, 10th: 1.67, 30th: 2.94, 50th: 4.19, 70th: 5.76, 90th: 8.53, 95th: 10.30, 99th: 13.90
bonus term 0.03700146960722674 // variance stats 0.4345298385021097 //part1 3.017088168272582 //part2 0.01226396695871561
-2.2648392152975774 3.1208759750427917 2.6901121614843158
-2.3388421545120313 3.1208759750427917 2.616109222269862
---
Quantiles: 1th: 0.70, 5th: 1.24, 10th: 1.68, 30th: 3.04, 50th: 4.34, 70th: 6.15, 90th: 9.76, 95th: 11.93, 99th: 16.82
bonus term 0.049110648239198515 // variance stats 0.3731928448149398 //part1 3.017088168272582 //part2 0.016277498535058246
-2.282937018292237 1.3427354743029658 2.32

Quantiles: 1th: 0.79, 5th: 1.44, 10th: 2.08, 30th: 3.82, 50th: 5.54, 70th: 7.68, 90th: 11.94, 95th: 14.33, 99th: 19.71
bonus term 0.08214842955243823 // variance stats 0.2698728951928677 //part1 3.017088168272582 //part2 0.027227719234824976
-1.706451221475303 2.218072930009925 2.0076194597999795
-1.8707480805801793 2.218072930009925 1.8433226006951031
---
Quantiles: 1th: 0.76, 5th: 1.33, 10th: 1.82, 30th: 3.40, 50th: 4.87, 70th: 6.88, 90th: 10.55, 95th: 12.98, 99th: 17.67
bonus term 0.06982028892832712 // variance stats 0.2816185571195827 //part1 3.017088168272582 //part2 0.02314161371303324
-1.8357672863733727 1.4406313989541972 1.9092070040155915
-1.9754078642300268 1.4406313989541972 1.7695664261589374
---
Quantiles: 1th: 0.74, 5th: 1.38, 10th: 1.83, 30th: 3.39, 50th: 4.99, 70th: 6.96, 90th: 11.21, 95th: 13.56, 99th: 19.17
bonus term 0.07972809935482891 // variance stats 0.26113079807575573 //part1 3.017088168272582 //part2 0.02642551192014943
-1.838578417313163 1.9545378193671803 

# size stuff

# a  = .25, k= 4,n=250

In [10]:
num_params=4
nobs=250
a=.25 
c1,c2 = calc_c(nobs)
alphas = [0.005,.01,.025,.05]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,
                                          data_tuned_c=data_tuned_c,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3,shi])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.78, 5th: 1.32, 10th: 1.81, 30th: 3.23, 50th: 4.66, 70th: 6.41, 90th: 9.87, 95th: 11.70, 99th: 17.15
bonus term 0.04111503232343808 // variance stats 0.4518855146145082 //part1 3.017088168272582 //part2 0.013627388405748274
-2.3816981814863265 -1.1609009322960393 2.862644397911523
-2.463928246133203 -1.1609009322960393 2.7804143332646465
---
Quantiles: 1th: 0.68, 5th: 1.21, 10th: 1.63, 30th: 2.98, 50th: 4.24, 70th: 5.83, 90th: 9.15, 95th: 10.97, 99th: 14.76
bonus term 0.059895322337048446 // variance stats 0.28570256880970624 //part1 3.017088168272582 //part2 0.019852029173990364
-2.785247976772794 -1.2481577337588852 2.6269334093072008
-2.905038621446891 -1.2481577337588852 2.5071427646331035
---
Quantiles: 1th: 0.71, 5th: 1.33, 10th: 1.80, 30th: 3.23, 50th: 4.73, 70th: 6.75, 90th: 10.71, 95th: 13.25, 99th: 18.42
bonus term 0.054317831071744004 // variance stats 0.36939147497265784 //part1 3.017088168272582 //part2 0.018003395340893666
-2.8793042341059905 -0.161821377

Quantiles: 1th: 0.66, 5th: 1.21, 10th: 1.63, 30th: 2.95, 50th: 4.16, 70th: 5.70, 90th: 8.70, 95th: 10.64, 99th: 14.97
bonus term 0.04667118417004987 // variance stats 0.35027881375341163 //part1 3.017088168272582 //part2 0.015468949386643617
-2.2933662973659192 0.21822404769135784 2.6609113605704775
-2.386708665706019 0.21822404769135784 2.567568992230378
---
Quantiles: 1th: 0.77, 5th: 1.36, 10th: 1.77, 30th: 3.13, 50th: 4.43, 70th: 6.10, 90th: 9.43, 95th: 11.29, 99th: 15.73
bonus term 0.0408972153247484 // variance stats 0.43407021600957996 //part1 3.017088168272582 //part2 0.01355519396311308
-2.298671537121507 0.5322192846968218 2.665185639599102
-2.380465967771004 0.5322192846968218 2.583391208949605
---
Quantiles: 1th: 0.66, 5th: 1.19, 10th: 1.58, 30th: 2.79, 50th: 4.10, 70th: 5.72, 90th: 8.99, 95th: 10.96, 99th: 15.46
bonus term 0.06091722290124862 // variance stats 0.27605859403283195 //part1 3.017088168272582 //part2 0.02019073341702389
-2.2530525698307287 0.26554314453465794 2

Quantiles: 1th: 0.80, 5th: 1.39, 10th: 1.86, 30th: 3.30, 50th: 4.65, 70th: 6.60, 90th: 10.19, 95th: 12.25, 99th: 16.94
bonus term 0.048938981069888796 // variance stats 0.390925432805978 //part1 3.017088168272582 //part2 0.016220600241161846
-1.9698443203746987 -0.26981505312884624 2.2756642603261588
-2.067722282514476 -0.26981505312884624 2.1777862981863816
---
Quantiles: 1th: 0.61, 5th: 1.20, 10th: 1.60, 30th: 2.86, 50th: 4.07, 70th: 5.68, 90th: 8.62, 95th: 10.50, 99th: 14.95
bonus term 0.051516575579757785 // variance stats 0.31394698685872613 //part1 3.017088168272582 //part2 0.017074932088992724
-2.0032165024460946 -0.24108741446660964 2.4054111278994648
-2.10624965360561 -0.24108741446660964 2.302377976739949
---
Quantiles: 1th: 0.77, 5th: 1.37, 10th: 1.86, 30th: 3.26, 50th: 4.55, 70th: 6.24, 90th: 9.35, 95th: 11.22, 99th: 15.28
bonus term 0.03789464957119179 // variance stats 0.46511570949282055 //part1 3.017088168272582 //part2 0.012560007350692765
-2.030343084184394 -0.4048294

-2.0608189929906633 0.6933301534791773 1.8152736139227037
-2.1543562794589644 0.6933301534791773 1.7217363274544026
---
Quantiles: 1th: 0.70, 5th: 1.32, 10th: 1.78, 30th: 3.06, 50th: 4.35, 70th: 6.04, 90th: 9.25, 95th: 11.43, 99th: 15.52
bonus term 0.0502694497620118 // variance stats 0.3453996331914348 //part1 3.017088168272582 //part2 0.016661577971317062
-1.913282580460706 -0.10323043650388113 1.87851424000488
-2.01382147998473 -0.10323043650388113 1.7779753404808565
---
Quantiles: 1th: 0.83, 5th: 1.48, 10th: 2.03, 30th: 3.62, 50th: 5.16, 70th: 7.17, 90th: 11.01, 95th: 13.66, 99th: 18.04
bonus term 0.0541075868516327 // variance stats 0.3811875242983158 //part1 3.017088168272582 //part2 0.017933710860896623
-1.8973219205634917 1.283633602935453 1.9085238728223328
-2.0055370942667574 1.283633602935453 1.8003086991190673
---
Quantiles: 1th: 0.76, 5th: 1.46, 10th: 1.98, 30th: 3.53, 50th: 5.00, 70th: 6.92, 90th: 10.53, 95th: 12.59, 99th: 17.51
bonus term 0.052579500087403384 // variance

# a  = .25, k= 4,n=500

In [11]:
###### num_params=4
nobs=500
a=.25 
c1,c2 = calc_c(nobs)
alphas = [0.005,.01,.025,.05]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3,shi])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.70, 5th: 1.35, 10th: 1.80, 30th: 3.18, 50th: 4.56, 70th: 6.41, 90th: 9.92, 95th: 11.74, 99th: 16.15
bonus term 0.004721940173415215 // variance stats 0.3277905299224012 //part1 3.465724215775732 //part2 0.0013624685287771244
-2.8383296191449925 1.3607897448088602 2.8756462211170355
-2.8477734994918227 1.3607897448088602 2.8662023407702053
---
Quantiles: 1th: 0.72, 5th: 1.34, 10th: 1.85, 30th: 3.41, 50th: 4.89, 70th: 6.95, 90th: 10.64, 95th: 12.73, 99th: 17.88
bonus term 0.0062276739970980695 // variance stats 0.24842884964585762 //part1 3.465724215775732 //part2 0.0017969329379268369
-3.2050565045590895 0.6301298608263217 2.665939509811697
-3.2175118525532858 0.6301298608263217 2.653484161817501
---
Quantiles: 1th: 0.66, 5th: 1.16, 10th: 1.58, 30th: 2.74, 50th: 3.96, 70th: 5.37, 90th: 7.98, 95th: 9.49, 99th: 12.67
bonus term 0.004393006226220796 // variance stats 0.3523678992757826 //part1 3.465724215775732 //part2 0.001267557933843709
-2.8801986622702667 -1.264621044

-2.449024783143006 0.8081695248268499 2.3988430657279043
-2.459251338791703 0.8081695248268499 2.388616510079207
---
Quantiles: 1th: 0.82, 5th: 1.56, 10th: 2.14, 30th: 3.84, 50th: 5.43, 70th: 7.37, 90th: 11.36, 95th: 13.69, 99th: 18.74
bonus term 0.0032937684347243527 // variance stats 0.47011379221426214 //part1 3.465724215775732 //part2 0.0009503838821713947
-2.2855608196159287 -0.07744245826602791 2.3708671550858385
-2.2921483564853777 -0.07744245826602791 2.3642796182163894
---
Quantiles: 1th: 0.83, 5th: 1.55, 10th: 2.07, 30th: 3.63, 50th: 5.13, 70th: 7.01, 90th: 10.56, 95th: 12.57, 99th: 17.14
bonus term 0.0036822085826234635 // variance stats 0.4204738050731563 //part1 3.465724215775732 //part2 0.001062464395136321
-2.608302398029588 0.8470746117358411 2.57841471015082
-2.6156668151948352 0.8470746117358411 2.5710502929855727
---
Quantiles: 1th: 0.71, 5th: 1.29, 10th: 1.78, 30th: 3.21, 50th: 4.65, 70th: 6.44, 90th: 9.69, 95th: 11.73, 99th: 16.17
bonus term 0.00412092264608252 // 

-2.372584541207344 -0.21459114621300346 2.0943793015725056
-2.380469550323854 -0.21459114621300346 2.0864942924559955
---
Quantiles: 1th: 0.73, 5th: 1.19, 10th: 1.57, 30th: 2.83, 50th: 4.04, 70th: 5.65, 90th: 8.70, 95th: 10.43, 99th: 13.94
bonus term 0.005131658542459114 // variance stats 0.3015835966628944 //part1 3.465724215775732 //part2 0.0014806886592707424
-2.36695151517456 0.5471003003687531 2.3386905994148397
-2.3772148322594786 0.5471003003687531 2.3284272823299212
---
Quantiles: 1th: 0.69, 5th: 1.38, 10th: 1.84, 30th: 3.30, 50th: 4.66, 70th: 6.48, 90th: 9.61, 95th: 11.57, 99th: 15.69
bonus term 0.0047293923690563035 // variance stats 0.32727331931888076 //part1 3.465724215775732 //part2 0.0013646187851671643
-2.379045142881959 -0.15648117340137682 2.308710234522668
-2.388503927620071 -0.15648117340137682 2.2992514497845558
---
Quantiles: 1th: 0.70, 5th: 1.31, 10th: 1.75, 30th: 3.09, 50th: 4.39, 70th: 6.00, 90th: 9.18, 95th: 11.06, 99th: 14.93
bonus term 0.004294168579205684 /

-1.9701886287916663 1.0676683194600132 1.8864167128886629
-1.9786313053784552 1.0676683194600132 1.877974036301874
---
Quantiles: 1th: 0.73, 5th: 1.41, 10th: 1.95, 30th: 3.41, 50th: 4.97, 70th: 6.82, 90th: 10.36, 95th: 12.72, 99th: 17.48
bonus term 0.004230466070354367 // variance stats 0.36592352706818954 //part1 3.465724215775732 //part2 0.0012206586003287807
-2.019649256789437 0.1779976116601503 1.9243263716983139
-2.028110188930146 0.1779976116601503 1.9158654395576054
---
Quantiles: 1th: 0.83, 5th: 1.38, 10th: 1.89, 30th: 3.33, 50th: 4.64, 70th: 6.35, 90th: 9.53, 95th: 11.73, 99th: 15.72
bonus term 0.0038760128411560626 // variance stats 0.3994273614036007 //part1 3.465724215775732 //part2 0.0011183846722462006
-1.8760843265280551 -0.770493381800488 1.8604506500578046
-1.8838363522103674 -0.770493381800488 1.8526986243754924
---
Quantiles: 1th: 0.76, 5th: 1.30, 10th: 1.74, 30th: 3.06, 50th: 4.40, 70th: 5.99, 90th: 9.06, 95th: 11.22, 99th: 15.29
bonus term 0.0047229284348396135 // 